download data and unzip
`kaggle competitions download -c titanic`

In [18]:
import pandas as pd
from sklearn import model_selection 
from sklearn import preprocessing
from sklearn import linear_model
from sklearn import ensemble
from sklearn import metrics

In [3]:
train_data = pd.read_csv("./dataset/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
test_data = pd.read_csv("./dataset/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [6]:
# 看标签列分布情况
train_data['Survived'].value_counts()

0    549
1    342
Name: Survived, dtype: int64

根据分布情况，决定使用什么样的cross-validation， k-fold k取多大。使用什么metric来当衡量指标。
数据量并不多，分布没有非常的偏斜。
kfold? AUC?

In [10]:
# 看每一列分布情况
def first_look_data(train_data):
    for col in train_data.columns:
        print(f"Number of distinct elements for column '{col}': {train_data[col].nunique()}")
        print(f"Distinct values and their count for column '{col}':")
        print(train_data[col].value_counts())
        print("==="*20)

first_look_data(train_data)

Number of distinct elements for column 'PassengerId': 891
Distinct values and their count for column 'PassengerId':
1      1
599    1
588    1
589    1
590    1
      ..
301    1
302    1
303    1
304    1
891    1
Name: PassengerId, Length: 891, dtype: int64
Number of distinct elements for column 'Survived': 2
Distinct values and their count for column 'Survived':
0    549
1    342
Name: Survived, dtype: int64
Number of distinct elements for column 'Pclass': 3
Distinct values and their count for column 'Pclass':
3    491
1    216
2    184
Name: Pclass, dtype: int64
Number of distinct elements for column 'Name': 891
Distinct values and their count for column 'Name':
Braund, Mr. Owen Harris                     1
Boulos, Mr. Hanna                           1
Frolicher-Stehli, Mr. Maxmillian            1
Gilinski, Mr. Eliezer                       1
Murdlin, Mr. Joseph                         1
                                           ..
Kelly, Miss. Anna Katherine "Annie Kate"    1
McC

In [13]:
from sklearn import model_selection 

train_data['kfold'] = -1
train_data.sample(frac=1).reset_index(drop=True) # frac = 1是sample作用在哪个轴上，reset_index是需要把sample后的df重新换新的index https://stackoverflow.com/questions/71758460/effect-of-pandas-dataframe-sample-with-frac-set-to-1
# fetch labels
y = train_data.Survived.values
# initiate the kfold class from model_selection module
kf = model_selection.StratifiedKFold(n_splits=5)

for fold, (trn_, val_) in enumerate(kf.split(X=train_data, y=y)): # 填充新的 kfold 列
    train_data.loc[val_, 'kfold'] = fold 

train_data.to_csv("./dataset/train_folds.csv", index=False) # 保存划分好的数据集

In [14]:
train_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,kfold
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,4
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,4
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,4
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,4


做完k-fold以及选择完metric，下一步做feature engineering

先处理NA值

1. 对类别值，第一种做法：mapping到数值上，labelEncoding =>适用于树形模型
2. 对类别值，第二种做法：直接用onehot 都能用，比上面的能多用于线性模型 liner regression and SVM， 对于多维的可以用 SVD降维
3. 还有种做法：用embedding。

对于在训练集中出现很少，或者几乎不出现的，但是测试集中却有很多的类别，需要特殊考虑。例如将测试集加到一起做label encoding or onehot， 或者新加一类unknown。等等

特征选择
以及特种组合

k-fold 的作用是用来选择好的模型，确保模型在各种情况都有很好的泛化能力。比如直接按80%/20%划分，万一很难预测的都在20%里，就无法衡量模型的能力。

最终是将选择好的模型，用所有的数据去train，然后用来预测。
https://stats.stackexchange.com/questions/52274/how-to-choose-a-predictive-model-after-k-fold-cross-validation
https://scikit-learn.org/stable/modules/cross_validation.html

## 问题： 如何选择K？


In [36]:
from sklearn import preprocessing
from sklearn import linear_model
from sklearn import ensemble
from sklearn import metrics
from sklearn import linear_model
from sklearn import ensemble
from sklearn import metrics
from sklearn import preprocessing
from scipy import sparse
from sklearn import decomposition
import xgboost as xgb

def run(fold):
    train_data = pd.read_csv("./dataset/train_folds.csv")
    num_cols = [
        "Pclass",
        "Age",
        "SibSp",
        "Parch",
        "Fare"
    ]

    features = [
        f for f in train_data.columns if f not in ("kfold", "Survived", "PassengerId")
    ]

    for col in features:
        # do not encode the numerical columns
        if col not in num_cols:
            train_data[col] = train_data[col].astype(str).fillna("NONE")

    # initialize LabelEncoder for each feature column
    lbl = preprocessing.LabelEncoder()

    # now its time to label encode the features
    for col in features:
        if col not in num_cols:

            # fit label encoder on all data
            lbl.fit(train_data[col])
            # transform all the data
            train_data[col] = lbl.transform(train_data[col])

    # get training data using folds
    df_train = train_data[train_data.kfold != fold].reset_index(drop=True)
    # get validation data using folds
    df_valid = train_data[train_data.kfold == fold].reset_index(drop=True)

    # get training data
    x_train = df_train[features].values
    # get validation data
    x_valid = df_valid[features].values

    # initialize Logistic Regression model
    model = xgb.XGBClassifier(n_jobs=-1, max_depth=7, n_estimators=200)
    # fit model on training data (ohe)
    model.fit(x_train, df_train.Survived.values)
    # predict on validation data
    # we need the probability values as we are calculating AUC
    # we will use the probability of 1s
    valid_preds = model.predict_proba(x_valid)[:, 1]
    # get roc auc score
    auc = metrics.roc_auc_score(df_valid.Survived.values, valid_preds)
    # print auc
    print(f"Fold = {fold}, AUC = {auc}")

In [37]:
for i in range(0, 5):
    run(i)

Fold = 0, AUC = 0.8591567852437418
Fold = 1, AUC = 0.8430481283422461
Fold = 2, AUC = 0.9012032085561498
Fold = 3, AUC = 0.8684491978609625
Fold = 4, AUC = 0.8772769578513495


In [38]:
def run_all_data():
    train_data = pd.read_csv("./dataset/train.csv")
    test_data = pd.read_csv("./dataset/test.csv")
    num_cols = [
        "Pclass",
        "Age",
        "SibSp",
        "Parch",
        "Fare"
    ]

    features = [
        f for f in train_data.columns if f not in ("Survived", "PassengerId")
    ]

    for col in features:
        # do not encode the numerical columns
        if col not in num_cols:
            train_data[col] = train_data[col].astype(str).fillna("NONE")

    # initialize LabelEncoder for each feature column
    lbl = preprocessing.LabelEncoder()

    # now its time to label encode the features
    for col in features:
        if col not in num_cols:

            # fit label encoder on all data
            lbl.fit(train_data[col])
            # transform all the data
            train_data[col] = lbl.transform(train_data[col])

    # get training data using folds
    df_train = train_data.reset_index(drop=True)
    # get validation data using folds
    df_valid = test_data

    # get training data
    x_train = df_train[features].values
    # get validation data
    x_valid = df_valid[features].values

    # initialize Logistic Regression model
    model = xgb.XGBClassifier(n_jobs=-1, max_depth=7, n_estimators=200)
    # fit model on training data (ohe)
    model.fit(x_train, df_train.Survived.values)
    # predict on validation data
    # we need the probability values as we are calculating AUC
    # we will use the probability of 1s
    valid_preds = model.predict_proba(x_valid)
    # get roc auc score
    valid_preds.to_csv("./dataset/predict.csv")


In [43]:
train_data = pd.read_csv("./dataset/train.csv")
test_data = pd.read_csv("./dataset/test.csv")
num_cols = [
    "Pclass",
    "Age",
    "SibSp",
    "Parch",
    "Fare"
]

features = [
    f for f in train_data.columns if f not in ("Survived", "PassengerId")
]

for col in features:
    # do not encode the numerical columns
    if col not in num_cols:
        train_data[col] = train_data[col].astype(str).fillna("NONE")
        test_data[col] = test_data[col].astype(str).fillna("NONE")

# initialize LabelEncoder for each feature column
lbl = preprocessing.LabelEncoder()

# now its time to label encode the features
for col in features:
    if col not in num_cols:

        # fit label encoder on all data
        lbl.fit(train_data[col])
        # transform all the data
        train_data[col] = lbl.transform(train_data[col])

# initialize LabelEncoder for each feature column
lbl_test = preprocessing.LabelEncoder()

# now its time to label encode the features
for col in features:
    if col not in num_cols:

        # fit label encoder on all data
        lbl_test.fit(test_data[col])
        # transform all the data
        test_data[col] = lbl_test.transform(test_data[col])


# get training data using folds
df_train = train_data.reset_index(drop=True)
# get validation data using folds
df_valid = test_data

# get training data
x_train = df_train[features].values
# get validation data
x_valid = df_valid[features].values

# initialize Logistic Regression model
model = xgb.XGBClassifier(n_jobs=-1, max_depth=7, n_estimators=200)
# fit model on training data (ohe)
model.fit(x_train, df_train.Survived.values)
# predict on validation data
# we need the probability values as we are calculating AUC
# we will use the probability of 1s
valid_preds = model.predict(x_valid)


In [46]:
predict = pd.read_csv("./dataset/test.csv")
predict.loc[:, 'Survived'] = valid_preds
predict[["PassengerId", "Survived"]].to_csv("./dataset/predict.csv", index=False)